In [20]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "menotrainer")))

In [1]:

from langchain_groq import ChatGroq
from typing import List, Dict, Any
from config import Configuration
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage
from services.api.app.agent.helpers import last_user_text
from services.api.app.agent.state import PhilosoferState   
from langchain_core.prompts import ChatPromptTemplate
from services.api.app.domain.prompts import (
    EXTRACT_EXERCISE_CARD
)
import numpy as np
from motor.motor_asyncio import AsyncIOMotorClient
#from services.api.app.agent.tools import retrieve_exercise_from_user_prompt

In [3]:
class ExerciseList(BaseModel):
    
    exercises: List[str] = Field(
        default_factory=list,
        description="Unique, canonical exercise names as strings. Empty list if none."
    )

In [4]:
def get_chat_model(
        temperature: float = Configuration.MODEL_TEMPERATURE,
        model: str = Configuration.MODEL_NAME
) -> ChatGroq:
    
    groq_model = ChatGroq(
        api_key=Configuration.GROQ_API_KEY,
        temperature=temperature,
        model=model
    )

    return groq_model 

In [18]:


last_message = "I want to know what is the main target muscles for Squats and deadlifts, what is the difference between the two? what about hip thrust and hip thurst with dumbbells?" #already extract content through last_user_text function

chat_model = get_chat_model(
    temperature= 0.0, 
    model = Configuration.FAST_MODEL_NAME
    ).with_structured_output(ExerciseList)

prompt = ChatPromptTemplate.from_messages([
    ("system", EXTRACT_EXERCISE_CARD.prompt),
    ("human", last_message)
])      

response = await (prompt | chat_model).ainvoke({})
raw_list = response.exercises
unique_exercises = np.unique([ex.strip().lower() for ex in raw_list if ex.strip()])
    
print(unique_exercises)

['deadlifts' 'hip thrust' 'hip thrust with dumbbells' 'squats']


In [7]:
from motor.motor_asyncio import AsyncIOMotorClient

In [96]:
async def fetch_target_muscles(unique_exercises: List[str]) -> Dict[str, Any]:
    
    MISSING = "No Data Found"
    mapping: Dict[str, str] = {ex: MISSING for ex in unique_exercises}

    if not unique_exercises:
        return mapping

    client = AsyncIOMotorClient(Configuration.mongo_uri)
    db = client[Configuration.MONGO_DB_NAME]
    coll = db[Configuration.MONGO_LONG_TERM_COLLECTION_EXERCISE]

    # Faster: one query with $in instead of looping
    # is basically a dictionary made out the collection in Mongo DB
    for ex in unique_exercises:
        print(ex)
        pipeline = [
            {"$search": {
                "index": 'exercise_serach_fuzzy',  # your Atlas Search index name
                "text": {
                    "path": "Exercise",
                    "query": ex,
                    "fuzzy": {
                        "maxEdits": 1,        # 0=exact only, 1–2 allow typos
                        "prefixLength": 1,    # first N chars must match exactly (helps precision)
                        "maxExpansions": 50
                    }
                }
            }},
            {"$project": {
                "_id": 0,
                "Exercise": 1,
                "Target Muscle Group ": 1,
                "score": {"$meta": "searchScore"}
            }},
            {"$limit": 1}
        ]

        docs = await coll.aggregate(pipeline).to_list(length=2)
        
        if docs:
            
            best = docs[0]
            mapping[ex] = best.get("Target Muscle Group ", MISSING)

    return mapping

In [97]:
docs = await fetch_target_muscles(['not an exercise','Single Arm Kettlebell bottoms Up Turkish Sit Up'])

not an exercise
Single Arm Kettlebell bottoms Up Turkish Sit Up


In [98]:
docs

{'not an exercise': 'Abdominals',
 'Single Arm Kettlebell bottoms Up Turkish Sit Up': 'Abdominals'}

In [34]:
async def test_mongo_connection():
    client = AsyncIOMotorClient(Configuration.mongo_uri)
    try:
        result = await client.admin.command('ping')
        print('MongoDB connection successful:', result)
    except Exception as e:
        print('MongoDB connection failed:', e)

await test_mongo_connection()

MongoDB connection successful: {'ok': 1}


In [3]:
import pandas as pd


ModuleNotFoundError: No module named 'pandas'

In [4]:
async def fetch_collection_as_dataframe():
    client = AsyncIOMotorClient(Configuration.mongo_uri)
    db = client[Configuration.MONGO_DB_NAME]
    coll = db[Configuration.MONGO_LONG_TERM_COLLECTION_EXERCISE]
    cursor = coll.find({})
    docs = []
    async for doc in cursor:
        docs.append(doc)
    
    return docs

In [5]:
df = await fetch_collection_as_dataframe()

In [6]:
df

[{'_id': ObjectId('689ea38b9397f360c9dbdbb7'),
  'Exercise': 'Bodyweight Glute Bridge',
  'Short YouTube Demonstration': 'Video Demonstration',
  'In-Depth YouTube Explanation': 'Video Explanation',
  'Difficulty Level': 'Beginner',
  'Target Muscle Group ': 'Glutes',
  'Prime Mover Muscle': 'Gluteus Maximus',
  'Secondary Muscle': 'Biceps Femoris',
  'Tertiary Muscle': 'Erector Spinae',
  'Primary Equipment ': 'Bodyweight',
  '# Primary Items': 1,
  'Secondary Equipment': 'None',
  '# Secondary Items': 0,
  'Posture': 'Bridge',
  'Single or Double Arm': 'No Arms',
  'Continuous or Alternating Arms ': 'Continuous',
  'Grip': 'No Grip',
  'Load Position (Ending)': 'No Load',
  'Continuous or Alternating Legs ': 'Continuous',
  'Foot Elevation': 'No Elevation',
  'Combination Exercises': 'Single Exercise',
  'Movement Pattern #1': 'Hip Extension',
  'Plane Of Motion #1': 'Sagittal Plane',
  'Body Region': 'Lower Body',
  'Force Type': 'Unsorted*',
  'Mechanics': 'Compound',
  'Laterality

# Testing Nodes / Edges

In [1]:
import sys
import os

# Add the parent directory to sys.path so 'menotrainer' is discoverable
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
from services.api.app.agent.state import PhilosoferState
from services.api.app.agent.chains import get_trainer_chain
from services.api.app.agent.nodes import conversation_node
from services.api.app.agent.edges import should_retrieve_exercise_content
from services.api.app.agent.chains import get_trainer_chain

from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import MessagesState

## conversation node

In [3]:
state = PhilosoferState()

In [4]:
response = await conversation_node(state = state, config = RunnableConfig)
state["messages"] = response.get("messages", [])
state["messages"]
#conversation_chain  = get_trainer_chain()

[AIMessage(content="Welcome to our fitness journey together. I'm excited to help you achieve your health and wellness goals. What's on your mind today? Are you looking to create a personalized meal plan, design a workout routine, or perhaps discuss strategies for staying motivated and accountable? Let's get started and make progress towards a healthier, happier you. What's your first question or area of focus?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 277, 'total_tokens': 354, 'completion_time': 0.212058144, 'prompt_time': 0.0442249, 'queue_time': 0.212557248, 'total_time': 0.256283044}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--af492155-8132-449b-ba8b-8005bf73681d-0', usage_metadata={'input_tokens': 277, 'output_tokens': 77, 'total_tokens': 354})]

In [5]:
hm = HumanMessage(content="are deadlifts safe?")
msgs = state.get("messages", [])

In [6]:
msgs.append(hm)
state["messages"] = msgs  

In [7]:
state.get("messages", [])

[AIMessage(content="Welcome to our fitness journey together. I'm excited to help you achieve your health and wellness goals. What's on your mind today? Are you looking to create a personalized meal plan, design a workout routine, or perhaps discuss strategies for staying motivated and accountable? Let's get started and make progress towards a healthier, happier you. What's your first question or area of focus?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 277, 'total_tokens': 354, 'completion_time': 0.212058144, 'prompt_time': 0.0442249, 'queue_time': 0.212557248, 'total_time': 0.256283044}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--af492155-8132-449b-ba8b-8005bf73681d-0', usage_metadata={'input_tokens': 277, 'output_tokens': 77, 'total_tokens': 354}),
 HumanMessage(content='are deadlifts safe?', additional_kwargs

In [8]:

response = await conversation_node(state = state, config = RunnableConfig)
response

{'messages': [AIMessage(content="Deadlifts can be a safe and effective exercise when performed properly. However, like any exercise, they do come with some risk of injury, particularly to the lower back. The key to safe deadlifting is to use proper form and technique.\n\nTo minimize the risk of injury, it's essential to:\n\n1. **Start with a manageable weight**: Don't try to lift too much too soon. Gradually increase the weight as you build strength and confidence.\n2. **Maintain proper form**: Keep your back straight, engage your core, and lift with your legs and hips, rather than just your back.\n3. **Use a full range of motion**: Lift the weight from the floor to hip level, squeezing your glutes and pushing your hips back at the top of the movement.\n4. **Avoid rounding your back**: Keep your chest up and your shoulders down, avoiding any curvature of the spine.\n5. **Warm up and cool down**: Always warm up before deadlifting, and cool down afterwards to prevent muscle strain.\n\nSo

In [9]:
print(response['messages'][0].content)

Deadlifts can be a safe and effective exercise when performed properly. However, like any exercise, they do come with some risk of injury, particularly to the lower back. The key to safe deadlifting is to use proper form and technique.

To minimize the risk of injury, it's essential to:

1. **Start with a manageable weight**: Don't try to lift too much too soon. Gradually increase the weight as you build strength and confidence.
2. **Maintain proper form**: Keep your back straight, engage your core, and lift with your legs and hips, rather than just your back.
3. **Use a full range of motion**: Lift the weight from the floor to hip level, squeezing your glutes and pushing your hips back at the top of the movement.
4. **Avoid rounding your back**: Keep your chest up and your shoulders down, avoiding any curvature of the spine.
5. **Warm up and cool down**: Always warm up before deadlifting, and cool down afterwards to prevent muscle strain.

Some common injuries associated with deadlift

## Should retrieve conditional function


In [12]:
state = PhilosoferState()
hm = HumanMessage(content="are deadlifts safe?")
msgs = state.get("messages", [])
msgs.append(hm)
state["messages"] = msgs  

In [13]:
 state.get("messages", [])

[HumanMessage(content='are deadlifts safe?', additional_kwargs={}, response_metadata={})]

In [14]:
await should_retrieve_exercise_content(state)

False

In [150]:
state = PhilosoferState()

# create a human message
hm = HumanMessage(content="What muscles do squats target?")

# append safely to the messages list and write back into the state
msgs = state.get("messages", [])
msgs.append(hm)
state["messages"] = msgs   # or state.update({"messages": msgs}) if you prefer

# verify
last = state.get("messages", [])[-1]
print(type(last), last.content)

<class 'langchain_core.messages.human.HumanMessage'> What muscles do squats target?


In [151]:
# create a human message
system = SystemMessage(content="Quadriceps!")

# append safely to the messages list and write back into the state
msgs = state.get("messages", [])
msgs.append(system)
state["messages"] = msgs   # or state.update({"messages": msgs}) if you prefer

# verify
last = state.get("messages", [])[-1]
print(type(last), last.content)

<class 'langchain_core.messages.system.SystemMessage'> Quadriceps!


In [155]:
await conversation_node(state = state, config = RunnableConfig)


AttributeError: 'dict' object has no attribute 'get_exercises_summary'

In [191]:
state.get("messages", [])
type(state.get("messages", []))

list

In [194]:
type(state.get('messages',[]))

list

In [ ]:
state = PhilosoferState()
state["messages"] = ['HI!']

In [228]:


exercise_data = state.get("exercises_data", {
    "exercises_user_ask_about": [],
    "main_muscle_target": []
})

In [229]:
exercise_data

{'exercises_user_ask_about': ['test 1', 'test 3'],
 'main_muscle_target': ['test 2', 'test 4']}

In [230]:
exercise_data['exercises_user_ask_about'].append('test 5')
exercise_data['main_muscle_target'].append('test 6')

In [231]:
state['exercises_data'] = exercise_data

In [232]:
state['exercises_data']

{'exercises_user_ask_about': ['test 1', 'test 3', 'test 5'],
 'main_muscle_target': ['test 2', 'test 4', 'test 6']}

In [ ]:
if 'exercises_data' not in state:
    state['exercises_data'] = {
        "exercises_user_ask_about": ['test 1'],
        "main_muscle_target": ['test 2']
    }
else:
    state['exercises_data']['exercises_user_ask_about'].append('test 3')
    state['exercises_data']['main_muscle_target'].append('test 4')

{'messages': ['HI!']}

In [201]:
state['messages'].append('Hello again!')

In [205]:
state['exercises_data'] = {
        "exercises_user_ask_about": ['test 1'],
        "main_muscle_target": ['test 2']
}

In [206]:
state

{'messages': ['HI!', 'Hello again!'],
 'exercises_data': {'exercises_user_ask_about': ['test 1'],
  'main_muscle_target': ['test 2']}}